In [1]:
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

C:\Users\murug\Anaconda3\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\murug\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.IPBC74C7KURV7CB2PKT5Z5FNR3SIBV4J.gfortran-win_amd64.dll
C:\Users\murug\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.TXA6YQSD3GCQQC22GEQ54J2UDCXDXHWN.gfortran-win_amd64.dll
  stacklevel=1)


In [2]:

from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score

In [29]:
print(os.getcwd())  
os.chdir(r'C:\\Users\\murug\\Desktop\\Jey\\GMU\\Semester 3\\DAEN 690\\mimic-iii-clinical-database-1.4\\allcsv')


C:\Users\murug\Desktop\Jey\GMU\Semester 3\DAEN 690\mimic-iii-clinical-database-1.4\allcsv


In [4]:
NOTEEVENTS=pd.read_csv('NOTEEVENTS.csv',dtype={'ROW_ID':np.int32, 'SUBJECT_ID': np.int32,'HADM_ID': np.float64,
                                       'CHARTDATE':str,'STORETIME':str,'CHARTTIME':str,
                                       'STORETIME': str,'CATEGORY': str,'DESCRIPTION':str,'CGID':str,'ISERROR':str,
                                        'TEXT':str}, parse_dates=['CHARTDATE'])
DIAGNOSES_ICD=pd.read_csv('DIAGNOSES_ICD.csv',dtype={'ROW_ID':np.int32, 'SUBJECT_ID': np.int32,'HADM_ID': np.int32,
                                            'SEQ_NUM':  np.float64,'ICD9_CODE':str})

In [5]:
NOTEEVENTS.head()

,ROW_ID,SUBJECT_ID,HADM_ID,CHARTDATE,CHARTTIME,STORETIME,CATEGORY,DESCRIPTION,CGID,ISERROR,TEXT
0,174,22532,167853.0,2151-08-04,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2151-7-16**] Dischar...
1,175,13702,107527.0,2118-06-14,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2118-6-2**] Discharg...
2,176,13702,167118.0,2119-05-25,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2119-5-4**] D...
3,177,13702,196489.0,2124-08-18,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2124-7-21**] ...
4,178,26880,135453.0,2162-03-25,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2162-3-3**] D...


In [6]:
DIAGNOSES_ICD.head()

,ROW_ID,SUBJECT_ID,HADM_ID,SEQ_NUM,ICD9_CODE
0,1297,109,172335,1.0,40301
1,1298,109,172335,2.0,486
2,1299,109,172335,3.0,58281
3,1300,109,172335,4.0,5855
4,1301,109,172335,5.0,4254


In [28]:
print('Null Values in NOTEEVENTS :\n\n',NOTEEVENTS.isnull().sum(axis = 0))
print('\n')
print('Null Values in DIAGNOSES_ICD :','\n\n',DIAGNOSES_ICD.isnull().sum(axis = 0))

Null Values in NOTEEVENTS :

 ROW_ID               0
SUBJECT_ID           0
HADM_ID         231836
CHARTDATE            0
CHARTTIME       316566
STORETIME       836776
CATEGORY             0
DESCRIPTION          0
CGID            836776
ISERROR        2082294
TEXT                 0
dtype: int64


Null Values in DIAGNOSES_ICD : 

 ROW_ID         0
SUBJECT_ID     0
HADM_ID        0
SEQ_NUM       47
ICD9_CODE     47
dtype: int64


In [27]:

print('NOTEEVENTS Total HADM ID count : ', NOTEEVENTS.HADM_ID.count())
print('NOTEEVENTS Unique HADM ID count : ', NOTEEVENTS.HADM_ID.nunique())
print('\n')
print('DIAGNOSES_ICD Total HADM ID count : ' , DIAGNOSES_ICD.HADM_ID.count())
print('DIAGNOSES_ICD Unique HADM ID count : ', DIAGNOSES_ICD.HADM_ID.nunique())
print('\n')
print('DIAGNOSES_ICD Total ICD9 Code count : ' , DIAGNOSES_ICD.ICD9_CODE.count())
print('DIAGNOSES_ICD Unique ICD9 Code count : ', DIAGNOSES_ICD.ICD9_CODE.nunique())


NOTEEVENTS Total HADM ID count :  1851344
NOTEEVENTS Unique HADM ID count :  58361


DIAGNOSES_ICD Total HADM ID count :  651047
DIAGNOSES_ICD Unique HADM ID count :  58976


DIAGNOSES_ICD Total ICD9 Code count :  651000
DIAGNOSES_ICD Unique ICD9 Code count :  6984


In [39]:
len(NOTEEVENTS.TEXT[39932])

14739

In [10]:
NOTEEVENTS.TEXT[15]

"Admission Date:  [**2203-11-3**]              Discharge Date:   [**2203-11-12**]\n\nDate of Birth:  [**2161-11-27**]             Sex:   M\n\nService: MEDICINE\n\nAllergies:\nPenicillins\n\nAttending:[**First Name3 (LF) 348**]\nChief Complaint:\nShortness of breath, chest pain\n\nMajor Surgical or Invasive Procedure:\nPericardiocentesis\nEGD with biopsy\n\n\nHistory of Present Illness:\n41 y/o M w/ h/o HIV/AIDS (HIV dx 83, AIDS 92, last CD4 132, VL\n>100K [**10/2203**], RF IVDU), not currently on HAART, previous right\nsided bacterial endocarditis with residual 4+ TR, h/o prior MI\nin [**2193**], who presents from [**Hospital **] Hospital for emergent\nevaluation of pericardial tamponade.\n\nPatient was recently hospitalized at [**Hospital1 18**] for osteomyelitis of\nhis L-ankle s/p prior fall. Presented to ED with fevers and\nankle pain. Taken to OR by ortho and found to have neg\nbrefringent crystals c/w gout. Tissue/Bone cultures grew MSSA.\nPatient started on cefazolin. F/U MRI co

In [59]:
DIAGNOSES_ICD[['ICD9_CODE','HADM_ID']]

,ICD9_CODE,HADM_ID
0,40301,172335
1,486,172335
2,58281,172335
3,5855,172335
4,4254,172335
...,...,...
651042,20280,188195
651043,V5869,188195
651044,V1279,188195
651045,5275,188195


In [60]:
NOTEEVENTS[['TEXT','HADM_ID']]

,TEXT,HADM_ID
0,Admission Date: [**2151-7-16**] Dischar...,167853.0
1,Admission Date: [**2118-6-2**] Discharg...,107527.0
2,Admission Date: [**2119-5-4**] D...,167118.0
3,Admission Date: [**2124-7-21**] ...,196489.0
4,Admission Date: [**2162-3-3**] D...,135453.0
...,...,...
2083175,NPN\n\n\n#1 Infant remains in RA with O2 sats...,115637.0
2083176,"Neonatology\nDOL #5, CGA 36 weeks.\n\nCVR: Con...",115637.0
2083177,Family Meeting Note\nFamily meeting held with ...,115637.0
2083178,NPN 1800\n\n\n#1 Resp: [**Known lastname 2243*...,115637.0


In [63]:
NOTE_DIAGNOSES_2=pd.merge(NOTEEVENTS,DIAGNOSES_ICD[['HADM_ID','ICD9_CODE']],on='HADM_ID')


In [64]:
NOTE_DIAGNOSES_2.head()

,ROW_ID,SUBJECT_ID,HADM_ID,CHARTDATE,CHARTTIME,STORETIME,CATEGORY,DESCRIPTION,CGID,ISERROR,TEXT,ICD9_CODE
0,174,22532,167853.0,2151-08-04,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2151-7-16**] Dischar...,01193
1,174,22532,167853.0,2151-08-04,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2151-7-16**] Dischar...,4254
2,174,22532,167853.0,2151-08-04,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2151-7-16**] Dischar...,42731
3,174,22532,167853.0,2151-08-04,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2151-7-16**] Dischar...,2639
4,174,22532,167853.0,2151-08-04,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2151-7-16**] Dischar...,2762


In [69]:
NOTE_DIAGNOSES_2.ICD9_CODE.nunique()

6967

In [70]:
NOTE_DIAGNOSES_2.HADM_ID.nunique()

58361

In [71]:
NOTE_DIAGNOSES_2[['HADM_ID','TEXT','ICD9_CODE']]

,HADM_ID,TEXT,ICD9_CODE
0,167853.0,Admission Date: [**2151-7-16**] Dischar...,01193
1,167853.0,Admission Date: [**2151-7-16**] Dischar...,4254
2,167853.0,Admission Date: [**2151-7-16**] Dischar...,42731
3,167853.0,Admission Date: [**2151-7-16**] Dischar...,2639
4,167853.0,Admission Date: [**2151-7-16**] Dischar...,2762
...,...,...,...
25736043,175166.0,Neonatology\nBaby Girl [**Known lastname 1672*...,V053
25736044,175166.0,Neonatology\nBaby Girl [**Known lastname 1672*...,V290
25736045,175166.0,NPN NICU\nTerm female admitted to NICU for sep...,V3001
25736046,175166.0,NPN NICU\nTerm female admitted to NICU for sep...,V053


In [91]:
NOTE_DIAGNOSES_2.loc[NOTE_DIAGNOSES_2['ICD9_CODE'] == '01193']

,ROW_ID,SUBJECT_ID,HADM_ID,CHARTDATE,CHARTTIME,STORETIME,CATEGORY,DESCRIPTION,CGID,ISERROR,TEXT,ICD9_CODE
0,174,22532,167853.0,2151-08-04,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2151-7-16**] Dischar...,01193
8,170,22532,167853.0,2151-08-04,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2151-7-16**] Dischar...,01193
16,59795,22532,167853.0,2151-08-02,NaN,NaN,Echo,Report,NaN,NaN,PATIENT/TEST INFORMATION:\nIndication: Aortic ...,01193
24,59796,22532,167853.0,2151-07-19,NaN,NaN,Echo,Report,NaN,NaN,PATIENT/TEST INFORMATION:\nIndication: Endocar...,01193
32,106289,22532,167853.0,2151-08-03,NaN,NaN,ECG,Report,NaN,NaN,Atrial fibrillation with a slow ventricular re...,01193
40,106290,22532,167853.0,2151-07-17,NaN,NaN,ECG,Report,NaN,NaN,Atrial fibrillation with a slow ventricular re...,01193
48,106291,22532,167853.0,2151-07-16,NaN,NaN,ECG,Report,NaN,NaN,Atrial fibrillation with a slow ventricular re...,01193
56,106292,22532,167853.0,2151-07-16,NaN,NaN,ECG,Report,NaN,NaN,Atrial fibrillation with a rapid ventricular r...,01193
64,763295,22532,167853.0,2151-07-16,2151-07-16 05:01:00,NaN,Radiology,CHEST (PORTABLE AP),NaN,NaN,[**2151-7-16**] 5:01 AM\n CHEST (PORTABLE AP) ...,01193
72,764802,22532,167853.0,2151-08-04,2151-08-04 15:39:00,NaN,Radiology,CT ABDOMEN W/CONTRAST,NaN,NaN,[**2151-8-4**] 3:39 PM\n CT ABDOMEN W/CONTRAST...,01193
